In [4]:

!pip install pypandoc    texlive-generic-recommended --break-system-packages 

# Install required packages
!pip install pyDOE --break-system-packages 
!pip install tensorflow --break-system-packages 
!pip install matplotlib --break-system-packages 
!pip install numpy --break-system-packages 
!pip install scipy --break-system-packages 

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement texlive-generic-recommended (from versions: none)
ERROR: No matching distribution found for texlive-generic-recommended
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18168 sha256=a57046e6e2130d4352fbb65e7801c7aaac8ce77f2663ac1bc87656f3963eb9d1
  Stored in directory: /home/toru/.cache/pip/wheels/84/20/8c/8bd43ba42b0b6d39ace1219d6da1576e0dac81b12265c4762e
Successf

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.0.2 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writea

In [ ]:
!sudo apt-get install texlive-generic-recommended

!sudo apt-get install texlive texlive-xetex texlive-latex-extra pandoc  texlive-generic-recommended


# Physics-Informed Neural Networks (PINNs) Implementation

This notebook implements Physics-Informed Neural Networks (PINNs) for solving different types of differential equations:
1. Lorenz-1960 System
2. Harmonic Oscillator (1st and 2nd order)
3. Hard-Constrained PINNs

The implementation uses TensorFlow 2.x and is designed to run in Google Colab.

In [8]:
# Install required packages
!pip install pyDOE  --break-system-packages 

Defaulting to user installation because normal site-packages is not writeable


In [11]:
!pip install tensorflow --break-system-packages 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pyDOE import lhs
from scipy.integrate import solve_ivp
import time

# Set random seeds for reproducibility
np.random.seed(1234)
tf.random.set_seed(1234)

Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'tensorflow'

## Utility Functions

In [ ]:
def get_collocation_points(n_points, t_final):
    """Generate collocation points using Latin Hypercube Sampling"""
    t = lhs(1, n_points).flatten()
    t = t_final * t
    return tf.convert_to_tensor(t, dtype=tf.float32)

class LossHistory:
    """Track and plot training losses"""
    def __init__(self):
        self.total_losses = []
        self.de_losses = []
        self.ic_losses = []

    def update(self, total_loss, de_loss, ic_loss):
        self.total_losses.append(float(total_loss))
        self.de_losses.append(float(de_loss))
        self.ic_losses.append(float(ic_loss))

    def plot(self):
        plt.figure(figsize=(10, 6))
        plt.semilogy(self.total_losses, label='Total Loss')
        plt.semilogy(self.de_losses, label='DE Loss')
        plt.semilogy(self.ic_losses, label='IC Loss')
        plt.xlabel('Iterations')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.show()

## Base PINN Implementation

In [ ]:
class PINN(tf.keras.Model):
    """Base class for Physics-Informed Neural Networks"""
    def __init__(self, layers, activation='tanh'):
        super().__init__()
        self.network_layers = []
        for units in layers[1:-1]:
            self.network_layers.append(tf.keras.layers.Dense(units, activation=activation))
        self.network_layers.append(tf.keras.layers.Dense(layers[-1]))

    def call(self, x):
        for layer in self.network_layers:
            x = layer(x)
        return x

## Lorenz-1960 System Implementation

In [ ]:
class LorenzPINN(PINN):
    def __init__(self, layers=[1, 20, 20, 20, 20, 3]):
        super().__init__(layers)
        self.k = 1.0
        self.l = 2.0

    @tf.function
    def get_residuals(self, t):
        with tf.GradientTape() as tape:
            tape.watch(t)
            u = self(t)
            x, y, z = tf.split(u, 3, axis=1)

        du_dt = tape.gradient(u, t)
        dx_dt, dy_dt, dz_dt = tf.split(du_dt, 3, axis=1)

        k, l = self.k, self.l
        f_x = k*l*(1.0/(k**2 + l**2) - 1.0/k**2)*y*z - dx_dt
        f_y = k*l*(1.0/l**2 - 1.0/(k**2 + l**2))*x*z - dy_dt
        f_z = k*l**2*(1.0/k**2 - 1.0/l**2)*x*y - dz_dt

        return f_x, f_y, f_z

def train_lorenz(t_final, n_points=1000, n_iter=10000):
    # Initial conditions
    x0, y0, z0 = 1.0, 0.5, 1.0

    # Create model and optimizer
    model = LorenzPINN()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Generate collocation points
    t = get_collocation_points(n_points, t_final)
    t0 = tf.zeros((1, 1), dtype=tf.float32)

    history = LossHistory()

    for i in range(n_iter):
        with tf.GradientTape() as tape:
            # Compute residuals
            f_x, f_y, f_z = model.get_residuals(t)

            # Initial conditions loss
            u0 = model(t0)
            x0_pred, y0_pred, z0_pred = tf.split(u0, 3, axis=1)
            ic_loss = tf.reduce_mean((x0_pred - x0)**2 +
                                    (y0_pred - y0)**2 +
                                    (z0_pred - z0)**2)

            # Differential equation loss
            de_loss = tf.reduce_mean(f_x**2 + f_y**2 + f_z**2)

            # Total loss
            loss = de_loss + ic_loss

        # Gradient descent step
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if i % 100 == 0:
            history.update(loss, de_loss, ic_loss)
            print(f'Iteration {i}, Loss: {loss:.6f}')

    history.plot()
    return model

## Harmonic Oscillator Implementation

In [ ]:
class OscillatorPINN(PINN):
    def __init__(self, layers=[1, 20, 20, 20, 20, 1], second_order=False):
        super().__init__(layers)
        self.second_order = second_order
        self.m = 1.0
        self.k = 2.0

    @tf.function
    def get_residuals(self, t):
        if self.second_order:
            with tf.GradientTape() as tape2:
                tape2.watch(t)
                with tf.GradientTape() as tape1:
                    tape1.watch(t)
                    u = self(t)
                du_dt = tape1.gradient(u, t)
            d2u_dt2 = tape2.gradient(du_dt, t)

            return self.m * d2u_dt2 + self.k * u
        else:
            with tf.GradientTape() as tape:
                tape.watch(t)
                u = self(t)
                x, v = tf.split(u, 2, axis=1)

            du_dt = tape.gradient(u, t)
            dx_dt, dv_dt = tf.split(du_dt, 2, axis=1)

            f_x = dx_dt - v
            f_v = dv_dt + (self.k/self.m) * x

            return f_x, f_v

def train_oscillator(t_final, second_order=False, n_points=1000, n_iter=10000):
    # Initial conditions
    x0, v0 = 1.0, 1.0

    # Create model and optimizer
    if second_order:
        model = OscillatorPINN(layers=[1, 20, 20, 20, 20, 1], second_order=True)
    else:
        model = OscillatorPINN(layers=[1, 20, 20, 20, 20, 2])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Generate collocation points
    t = get_collocation_points(n_points, t_final)
    t0 = tf.zeros((1, 1), dtype=tf.float32)

    history = LossHistory()

    for i in range(n_iter):
        with tf.GradientTape() as tape:
            if second_order:
                # Second order implementation
                residual = model.get_residuals(t)
                de_loss = tf.reduce_mean(residual**2)

                # Initial conditions
                with tf.GradientTape() as tape_ic:
                    tape_ic.watch(t0)
                    u0 = model(t0)
                du0_dt = tape_ic.gradient(u0, t0)

                ic_loss = tf.reduce_mean((u0 - x0)**2 + (du0_dt - v0)**2)
            else:
                # First order system implementation
                f_x, f_v = model.get_residuals(t)
                de_loss = tf.reduce_mean(f_x**2 + f_v**2)

                # Initial conditions
                u0 = model(t0)
                x0_pred, v0_pred = tf.split(u0, 2, axis=1)
                ic_loss = tf.reduce_mean((x0_pred - x0)**2 + (v0_pred - v0)**2)

            # Total loss
            loss = de_loss + ic_loss

        # Gradient descent step
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if i % 100 == 0:
            history.update(loss, de_loss, ic_loss)
            print(f'Iteration {i}, Loss: {loss:.6f}')

    history.plot()
    return model

## Hard-Constrained PINN Implementation

In [ ]:
class HardConstrainedPINN(PINN):
    """Hard-constrained Physics-Informed Neural Network"""
    def __init__(self, layers=[1, 20, 20, 20, 20, 3], initial_conditions=None):
        super().__init__(layers)
        self.initial_conditions = initial_conditions or {}

    def call(self, t):
        # Base network output
        N = super().call(t)

        # Apply hard constraints using t·N(t) formulation
        outputs = []
        for i, (name, ic) in enumerate(self.initial_conditions.items()):
            y = ic + t * N[:, i:i+1]
            outputs.append(y)

        return tf.concat(outputs, axis=1)

class TaylorHardConstrainedPINN(PINN):
    """Second-order Taylor expansion based hard-constrained PINN"""
    def __init__(self, layers=[1, 20, 20, 20, 20, 1], u0=0.0, v0=0.0):
        super().__init__(layers)
        self.u0 = u0
        self.v0 = v0

    def call(self, t):
        # Base network output
        N = super().call(t)

        # Second-order Taylor expansion
        y = self.u0 + self.v0 * t + t**2 * N
        return y

def train_hard_constrained_lorenz(t_final, n_points=1000, n_iter=10000):
    x0, y0, z0 = 1.0, 0.5, 1.0

    # Create model with hard constraints
    model = HardConstrainedPINN(initial_conditions={'x': x0, 'y': y0, 'z': z0})
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Generate collocation points
    t = get_collocation_points(n_points, t_final)

    history = LossHistory()

    for i in range(n_iter):
        with tf.GradientTape() as tape:
            f_x, f_y, f_z = model.get_residuals(t)
            de_loss = tf.reduce_mean(f_x**2 + f_y**2 + f_z**2)
            loss = de_loss  # No IC loss needed for hard constraints

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if i % 100 == 0:
            history.update(loss, de_loss, 0.0)
            print(f'Iteration {i}, Loss: {loss:.6f}')

    history.plot()
    return model

## Example Usage and Comparison

In [ ]:
# Compare different integration times for Lorenz system
tf_values = [1, 5, 10, 20]
results = {}

for tfval in tf_values:
    print(f'\nTraining for tf = {tfval }')
    # Train soft-constrained PINN
    model_soft = train_lorenz(tfval )

    # Train hard-constrained PINN
    model_hard = train_hard_constrained_lorenz(tfval )

    # Classical solver (reference solution)
    def lorenz_system(t, y):
        x, y, z = y
        k, l = 1.0, 2.0
        dx = k*l*(1.0/(k**2 + l**2) - 1.0/k**2)*y*z
        dy = k*l*(1.0/l**2 - 1.0/(k**2 + l**2))*x*z
        dz = k*l**2*(1.0/k**2 - 1.0/l**2)*x*y
        return [dx, dy, dz]

    sol = solve_ivp(lorenz_system, [0, tf], [1.0, 0.5, 1.0],
                   method='RK45', rtol=1e-8, atol=1e-8)

    # Evaluate solutions on uniform grid
    t_eval = np.linspace(0, tf, 1000).reshape(-1, 1)
    t_eval_tf = tf.convert_to_tensor(t_eval, dtype=tf.float32)

    y_soft = model_soft(t_eval_tf).numpy()
    y_hard = model_hard(t_eval_tf).numpy()

    results[tf] = {
        't': t_eval,
        'soft': y_soft,
        'hard': y_hard,
        'classical': np.array([np.interp(t_eval.flatten(), sol.t, sol.y[i])
                              for i in range(3)]).T
    }

In [ ]:
def get_collocation_points(n_points, t_final):
    """Generate collocation points using Latin Hypercube Sampling"""
    t = lhs(1, n_points).flatten()
    t = t_final * t
    # Reshape t to have an extra dimension for the batch size
    t = t.reshape(-1, 1)  # Reshape to (n_points, 1)
    return tf.convert_to_tensor(t, dtype=tf.float32)

In [ ]:
import numpy as np
import tensorflow as tf
from scipy.integrate import solve_ivp

# Compare different integration times for Lorenz system
tf_values = [1, 5, 10, 20]
results = {}

for t_final in tf_values:  # Renamed tfval to t_final to avoid conflicts
    print(f'\nTraining for tf = {t_final}')

    # Train soft-constrained PINN
    model_soft = train_lorenz(t_final)

    # Train hard-constrained PINN
    model_hard = train_hard_constrained_lorenz(t_final)

    # Classical solver (reference solution)
    def lorenz_system(t, y):
        x, y, z = y
        k, l = 1.0, 2.0
        dx = k * l * (1.0 / (k**2 + l**2) - 1.0 / k**2) * y * z
        dy = k * l * (1.0 / l**2 - 1.0 / (k**2 + l**2)) * x * z
        dz = k * l**2 * (1.0 / k**2 - 1.0 / l**2) * x * y
        return [dx, dy, dz]

    sol = solve_ivp(lorenz_system, [0, t_final], [1.0, 0.5, 1.0],
                    method='RK45', rtol=1e-8, atol=1e-8)

    # Evaluate solutions on uniform grid
    t_eval = np.linspace(0, t_final, 1000).reshape(-1, 1)
    t_eval_tf = tf.convert_to_tensor(t_eval, dtype=tf.float32)  # Ensure tf is TensorFlow

    y_soft = model_soft(t_eval_tf).numpy()
    y_hard = model_hard(t_eval_tf).numpy()

    results[t_final] = {
        't': t_eval,
        'soft': y_soft,
        'hard': y_hard,
        'classical': np.array([np.interp(t_eval.flatten(), sol.t, sol.y[i])
                               for i in range(3)]).T
    }


In [ ]:
# Visualization of results
def plot_comparison(tf):
    res = results[tf]
    fig = plt.figure(figsize=(15, 5))
    titles = ['x(t)', 'y(t)', 'z(t)']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.plot(res['t'], res['classical'][:, i], 'k-', label='Classical')
        plt.plot(res['t'], res['soft'][:, i], 'r--', label='Soft PINN')
        plt.plot(res['t'], res['hard'][:, i], 'b:', label='Hard PINN')
        plt.xlabel('t')
        plt.ylabel(titles[i])
        plt.title(f'{titles[i]} (tf={tf})')
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()

# Plot results for each tf
for tf in tf_values:
    plot_comparison(tf)

# Compute and display errors
for tf in tf_values:
    res = results[tf]
    err_soft = np.mean((res['soft'] - res['classical'])**2)
    err_hard = np.mean((res['hard'] - res['classical'])**2)
    print(f'\nMean squared error for tf={tf}:')
    print(f'Soft PINN: {err_soft:.2e}')
    print(f'Hard PINN: {err_hard:.2e}')